In [1]:
# add ieml library to kernel path
import sys
sys.path.insert(0, '..')

The ieml library exposes a parser (ieml.usl.parser) that parses ieml strings (including older versions) and return a normalised ieml.usl.USL object or a ieml.dictionary.Script (a morpheme). For the moment only ieml.usl.Word are valid ieml.usl.USL.

The ieml.usl.USL can be checked for coherancy with ieml.usl.USL.check(). That feature should be added in a strict mode for the parser.

In [5]:
from ieml.usl.usl import usl
u = usl("[E:.b.E:B:.- E:S:. (E:.-wa.-t.o.-' E:.-'wu.-S:.-'t.o.-',)(a.T:.-) > ! E:.l.- (E:.wo.- E:S:.-d.u.-')]")
u.check()
print(u)
u1 = usl("[E:.b.E:B:.- E:S:. (E:.-'wu.-S:.-'t.o.-', E:.-wa.-t.o.-' )(a.T:.-) > ! E:.l.- (E:.wo.- E:S:.-d.u.-')]")
u1.check()
print(u1)
assert u1 == u

[E:.b.E:B:.- E:S:. (E:.-wa.-t.o.-' E:.-'wu.-S:.-'t.o.-',)(a.T:.-) > ! E:.l.- (E:.wo.- E:S:.-d.u.-')]
[E:.b.E:B:.- E:S:. (E:.-wa.-t.o.-' E:.-'wu.-S:.-'t.o.-',)(a.T:.-) > ! E:.l.- (E:.wo.- E:S:.-d.u.-')]


The ieml lexicons are stored on github, they have to be downloaded first

In [7]:
from ieml.ieml_database import GitInterface, IEMLDatabase
gitdb = GitInterface()
gitdb.pull() # download database in ~/.cache/ieml/ folder
print(gitdb)

# instanciate a ieml.ieml_database.IEMLDatabase from the downloaded git repository
db = IEMLDatabase(folder=gitdb.folder)
print(db)

<ieml.ieml_database.ieml_database (/home/louis/.cache/ieml/1.0.3/e3050257e00f171aa548e17e73fe562e cache=/home/louis/.cache/ieml/1.0.3/e3050257e00f171aa548e17e73fe562e)>


The ieml.ieml_database.IEMLDatabase is responsible of reading and writing association between ieml.usl.USL and ieml.dictionary.Script with their translation to disk. The values as stored in rows in one file per USL. The format of the rows are space separated values (ssv like csv or tsv).


# ieml.dictionary.Script

The Script is a old name for morphemes. The 

# Words

In [16]:
# ou se trouve le pointd'exclamation
str(u.role)

'E:.l.-'

In [24]:
print('\n'.join(str(r) + ' ' + str(a.actor) for r, a in u.syntagmatic_fun.actors.items()))

E:T:. (E:.-wa.-t.o.-' E:.-'wu.-S:.-'t.o.-',)(a.T:.-)
E:.l.- (E:.wo.- E:S:.-d.u.-')


In [29]:
from ieml.usl.constants import ADDRESS_SCRIPTS,NAMES_TO_ADDRESS 

# list des roles syntagmatic
print('\n'.join(str(r) + ' ' + NAMES_TO_ADDRESS[r] for r in ADDRESS_SCRIPTS))

E:S:. process
E:T:. process
E:B:. process
E:.n.- initiator
E:.d.- interactant
E:.k.- recipient
E:.t.- time
E:.l.- location
E:.f.- manner
E:.s.- cause
E:.m.- intention
E:A:. dependant
E:U:. independant


In [35]:
# parse all words
usls = db.list(parse=False, type='word')
usls

["[E:.b.E:B:.- ! E:.s.- (E:.wo.- E:S:.-d.u.-')(E:T:.x.- s.e.-s.u.-')]",
 "[E:.b.E:B:.- ! E:.f.- (E:.wo.- E:.A:.c.- E:.-n.S:.-' E:S:.-d.u.-')(f.a.-m.a.-')]",
 "[E:.b.E:B:.- ! E:.f.- (E:.wo.- E:S:.-d.u.-')(k.o.-t.o.-')]",
 "[E:.b.E:B:.- ! E:A:. (E:.wo.- E:S:.-d.u.-')(E:S:T:.)]",
 "[E:.b.E:B:.- ! E:.f.- (E:.wo.- E:S:.-d.u.-')(wu.wu.t.-)]",
 "[E:.b.E:B:.- ! E:.l.- (E:.wo.- E:.-U:.f.-l.-' E:S:.-d.u.-')(k.i.-l.i.-')]",
 "[E:.b.E:B:.- ! E:.l.- (E:.wo.- E:.-U:.t.-l.-' E:S:.-d.u.-')(k.i.-l.i.-')]",
 "[E:.b.E:B:.- ! E:.f.- (E:.wo.- E:S:.-d.u.-')(t.o.-k.o.-')]",
 "[E:.b.E:B:.- ! E:A:. (E:.wo.- E:S:.-d.u.-')(m.-T:.U:.-') > E:A:. E:U:. (E:B:.-d.u.-')(p.E:A:T:.-)]",
 "[E:.b.E:B:.- E:B:. (E:.-wa.-t.o.-' E:.-'we.-S:.-'t.o.-',)(i.k.-) > ! E:.n.- (E:.wo.- E:.-n.S:.-' E:S:.-d.u.-')]",
 "[E:.b.E:B:.- ! E:A:. (E:.wo.- E:S:.-d.u.-')(E:.A:.j.-)]",
 "[E:.b.E:B:.- E:S:. (E:.-wa.-t.o.-' E:.-'wu.-S:.-'t.o.-',)(u.A:.-) > ! E:.n.- (E:.wo.- E:S:.-d.u.-')(l.-T:.U:.-',n.-T:.A:.-',t.o.-f.o.-',_) > E:.l.- (E:.wo.- E:.-

In [3]:
from ieml.usl import USL
from ieml.dictionary.script import Script

# the database contains the morphemes and the usls made from morphemes
all(isinstance(u, (USL, Script)) for u in usls) 

True

In [35]:
descriptorsDB = db.get_descriptors()
def display_usls(u):
    descriptor = descriptorsDB.get_values_partial(u)
    # descriptor is a dict :
    # { (ieml, descriptor_type, lang) : string[]}
    # descriptor_type in ieml.constants.DESCRIPTORS_CLASS = ['translations', 'comments', 'tags']
    # lang in ieml.constants.LANGUAGES = [ 'fr', 'en' ]  
    
    return str(u) +\
        "".join("\n\t{}: {}".format(k, str(v)) for k, v in {
            **{'{}_{}'.format(descriptor_type, lang): ', '.join(t) 
               for (_, descriptor_type, lang), t in descriptor.items()},
            'type': u.__class__.__name__,
            'is_paradigm': not u.is_singular,
            
        }.items())

In [36]:
from ieml.usl.usl import usl

# usl() parse the string and return an Usl or a Script (if a morpheme)
oo_script = usl('O:O:.')
assert isinstance(oo_script, Script)

print(display_usls(usl('O:O:.')))

O:O:.
	en_comments: In order to understand O x O we have to think about the basic sensorimotor cycle. Let's begin with the inner sanctum of personal existence that does not manifest externally: to exist. Then, from this virtual place, we act. As a result, our actual environment is transformed and a new reality appears or happens. Finally, from the actuality of external happening, we perceive back to our inner existence. Note the two intertwined oppositions : \n\n• wa act (in –> out) / wu perceive (out –> in) \n\n• wo exist (in –> in) / we appear (out –> out)\n\nwo wa wu we can be used to model any quaternary semantic symmetry. They can also be used to « verbify » nominal semes when creating new words.
	en_translations: interaction phases, tetrad
	fr_translations: phases de l'interaction, tétrade
	type: MultiplicativeScript
	is_paradigm: True
